# CONSTANTS USED FOR GENERATION. FEEL FREE TO EDIT.
To edit outputs, scroll down until you find the next massive header.

In [ ]:
# What types of areas different rooms can be
AREA_TYPES = ["building", "floor", "room"]
# The types of features rooms can include
FEATURES = ["Screen", "Camera", "Microphone"]

# How many users to generate
USER_COUNT = 10
# How many areas to generate
AREA_COUNT = 20
# How many bookings to generate. Note that these may be silently
# dropped if they overlap with existsing bookings.
MAX_BOOKING_COUNT = 1000000

***
Code below
***

In [4]:
import random
import string
from datetime import datetime, time, date, timedelta

def rand_text(length: int=None, whitespace: bool = False) -> str:
	text = string.ascii_letters
	if whitespace:
		text = text + " "
	if (not length):
		length = random.randint(3, 20)
	text = "".join(random.choices(text, k=length)).capitalize();
	return text

def rand_text_in(min: int = 5, max: int = 12, whitespace: bool = False) -> str:
	return rand_text(random.randint(min, max), whitespace)

def rand_bool(odds=0.5) -> bool:
	return random.random() < odds

def rand_timedelta(duration=128):
	return timedelta(
		hours=random.randint(0, duration - 1),
		minutes=random.randint(0, 59),
		seconds=random.randint(0, 59)
	)

def choices_x(arr: list, num: int = 3) -> list:
	arr = arr.copy()
	retval = []
	for _ in range(num):
		idx = random.randint(0, len(arr) - 1)
		retval.append(arr.pop(idx))
	return retval

In [5]:
import uuid
from uuid import UUID
class IdTicker:
	def get(self) -> uuid.UUID:
		newId = uuid.uuid4()

		return newId

In [6]:
import requests
def validate_response(response: requests.Response):
	if not response.ok:
	 response.raise_for_status()

In [7]:

SERVER = "http://localhost:8080"
SERVER_AREA_TYPE = SERVER + "/area-type"
SERVER_AREA_FEATURE = SERVER + "/area-feature"
SERVER_USER = SERVER + "/user"
SERVER_AREA = SERVER + "/area"
SERVER_BOOKING = SERVER + "/reservation"

# Area type and feature setup

In [8]:
for area_type in AREA_TYPES:
	if not requests.get(SERVER_AREA_TYPE + f"/{area_type}").ok:
		print("posting")
		requests.post(SERVER_AREA_TYPE, json={"name": area_type, "description": ""})

In [9]:
for feature in FEATURES:
	if not requests.get(SERVER_AREA_FEATURE + f"/{feature}").ok:
		print("posting")
		requests.post(SERVER_AREA_FEATURE, json={"name": feature, "description": ""})

In [10]:
def get_user_ids():
	users = requests.get(SERVER_USER).json()
	user_ids = []
	for user in users:
		user_ids.append(user.get("id"))
	print(f"Found {len(user_ids)} users")
	return user_ids
user_ids = get_user_ids()

Found 1 users


In [11]:
def get_area_ids():
	areas_page = requests.get(SERVER_AREA + "?page=0").json()
	areas = areas_page.get("content")
	for page in range(1, areas_page.get("totalPages")):
		this_page = requests.get(SERVER_AREA + f"?page={page}").json()
		areas = [*areas, *this_page.get("content")]
	area_ids = []
	for area in areas:
		area_ids.append(area.get("id"))
	print(f"Found {len(area_ids)} areas")
	return area_ids
area_ids = get_area_ids()

Found 5 areas


# USER

In [ ]:
class User:
	id: UUID
	firstName: str = "ERROR"
	lastName: str = "ERROR"
	email: str = "ERROR@ERROR.ERR"

	def __init__(self):
		self.firstName = rand_text()
		self.lastName = rand_text()
		self.email = rand_text() + "@" + rand_text() + "." + rand_text_in(2, 3)

	def get_json(self):
		return {
			"name": self.firstName + self.lastName,
			"email": self.email
		}

In [ ]:
for i in range(USER_COUNT):
	response = requests.post(SERVER_USER, json=User().get_json())
	validate_response(response)

# AREAS

In [41]:
class Area:
	administratorIds: list[UUID] = []
	superAreas: list = []
	areaTypeId: str
	areaFeatureIds: list[str] = []
	capacity: int
	calendarLink: str
	name: str
	description: str
	reservable: bool = True
	id: str = None

	def __init__(self):
		self.administratorIds = [random.choice(user_ids)]
		self.superAreas = []
		self.areaTypeId = {"name": random.choice(AREA_TYPES), "description": ""}
		self.areaFeatureIds = choices_x(FEATURES, random.randint(1,3))
		self.areaFeatureIds = [{"name": feat, "description": ""} for feat in self.areaFeatureIds]
		self.capacity = random.randint(1,10)
		self.name = rand_text()
		self.description = rand_text_in(70, 150, whitespace=True)

	def get_json(self):
		superAreas = []
		for area in self.superAreas:
			superAreas.append(area.get_json())
		return {
			"id": self.id,
			"administratorIds": self.administratorIds,
			"superAreas": superAreas,
			"areaType": self.areaTypeId,
			"areaFeatures": self.areaFeatureIds,
			"capacity": self.capacity,
			"name": self.name,
			"description": self.description,
			"reservable": self.reservable
		}

In [ ]:
# Refresh area ids
area_ids = get_area_ids()
for i in range(AREA_COUNT):
	response = requests.post(SERVER_AREA, json=Area().get_json())
	validate_response(response)

# RESERVATIONS

In [71]:
class Reservation:
	user: UUID
	area: UUID
	startTime: datetime
	endTime: datetime
	comment: str

	def __init__(self):
		self.user = random.choice(user_ids)
		self.area = random.choice(area_ids)

		current_time = datetime.now()
		self.startTime = current_time + rand_timedelta(1024)
		self.endTime = self.startTime + rand_timedelta(8)

		self.comment = rand_text_in(70, 150, whitespace=True)

	def get_json(self):
		return {
			"userId": self.user,
			"areaId": self.area,
			"startTime": self.startTime.strftime("%Y-%m-%dT%H:%M:%S"),
			"endTime": self.endTime.strftime("%Y-%m-%dT%H:%M:%S"),
			"comment": self.comment
		}

In [73]:
# Refresh IDs
user_ids = get_user_ids()
area_ids = get_area_ids()

count = 0

session = requests.Session()
for i in range(MAX_BOOKING_COUNT):
	response = session.post(SERVER_BOOKING, json=Reservation().get_json())
	if not response.ok and response.status_code != 409:
		pass # fuck you
		# response.raise_for_status()
	elif response.status_code != 409:
		count += 1;

f"Created {count} reservations"

Found 1 users
Found 488 areas


'Created 9880 reservations'

# Building Structure

In [55]:
session = requests.Session()
def post_area(area: Area) -> str:
	response = session.post(SERVER_AREA, json=area.get_json())
	response.raise_for_status()
	return response.text.strip('"')

In [56]:
class Room:
	name: str
	floorNum: int
	letter: str

	def __init__(self, letter: str, floorNum: int, roomNum: int):
		self.floorNum = floorNum
		self.letter = letter
		self.name = f"{letter}{floorNum}{str(roomNum).zfill(2)}"

	def get_area(self):
		area = Area()
		area.name = self.name
		area.description = f"A room on floor {self.floorNum} of building {self.letter}"
		area.areaTypeId = "room"
		return area

	def post_self(self, parent_id: str) -> str:
		area = self.get_area()

		parent_area = Area()
		parent_area.id = parent_id
		area.superAreas.append(parent_area)

		return post_area(area)

In [ ]:
class Floor:
	name: str
	letter: str
	floorNum: int
	rooms: list[Room] = []
	roomCount: int

	def __init__(self, letter, floorNum):
		self.rooms = []
		self.roomCount = random.randint(5,30)
		self.name = f"{letter}{str(floorNum).zfill(2)}"
		self.letter = letter
		self.floorNum = floorNum

		for i in range(self.roomCount):
			self.rooms.append(Room(letter, floorNum, i))

	def get_area(self):
		area = Area()
		area.name = self.name
		area.description = f"A floor in building {self.letter}"
		area.areaTypeId = "floor"
		return area

	def post_self(self, parent_id: str) -> str:
		print(f"Posting floor {self.name}")
		area = self.get_area()

		parent_area = Area()
		parent_area.id = parent_id
		area.superAreas.append(parent_area)

		floor_id = post_area(area)

		for room in self.rooms:
			room.post_self(floor_id)

		return floor_id

In [ ]:
class Building:
	letter: str
	floorCount: int
	floors: list[Floor] = []

	def __init__(self, letter):
		self.floors = []
		self.letter = letter
		self.floorCount = random.randint(1, 9)

		for i in range(self.floorCount):
			self.floors.append(Floor(letter, i + 1))

	def get_area(self):
		area = Area()
		area.name = self.letter
		area.description = f"This is building {self.letter}"
		area.reservable = False
		area.areaTypeId = "building"
		return area

	def post_self(self):
		print(f"Posting building {self.letter}")
		building_id = post_area(self.get_area())

		for floor in self.floors:
			floor.post_self(building_id)

		return building_id



In [70]:
buildingLetters = ["A", "B", "C", "D", "E", "F"]
for buildingLetter in buildingLetters:
	print(buildingLetter)
	building = Building(buildingLetter)
	building.post_self()


A
A
Posting A
Posting A01
Posting A02
B
B
Posting A
Posting B01
C
C
Posting A
Posting C01
Posting C02
Posting C03
D
D
Posting A
Posting D01
Posting D02
Posting D03
E
E
Posting A
Posting E01
Posting E02
Posting E03
Posting E04
Posting E05
F
F
Posting A
Posting F01
Posting F02
Posting F03
Posting F04
Posting F05
Posting F06
Posting F07
Posting F08
